In [6]:
from PIL import Image
from abc import ABC, abstractmethod
from typing import TypeVar, Generic, Any, Dict, Type, Optional
from pydantic import BaseModel
from pydantic_core import from_json

T = TypeVar("T")


class BaseAction(BaseModel, ABC, Generic[T]):
    """Action performed by the agent."""

    preaction_screenshot: Optional[str] = None
    postaction_screenshot: Optional[str] = None

    @property
    @abstractmethod
    def engine() -> str:
        pass

    @property
    @abstractmethod
    def args() -> T:
        pass


class Action(BaseAction[Any]):
    pass


class ActionParser(BaseModel):
    engines: Dict[str, Type[BaseAction]] = {}

    def add_engine(self, engine: str, action: Type[BaseAction]):
        self.engines[engine] = action

    def remove_engine(self, engine: str):
        if engine in self.engines:
            del self.engines[engine]

    def parse(self, action_json: str | bytes | bytearray) -> BaseAction:
        data = from_json(action_json)
        engine = data.get("engine")

        return self.engines.get(engine, Action)(**data)


DEFAULT_PARSER = ActionParser()